## Titanic FL

In [1]:
import pandas as pd
import numpy as np 
from preprocess import Preprocessor
import os

In [2]:
df = pd.read_csv('data/titanic.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Age,Sex,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,22.0,male,1,0,7.2500,N,S
1,2,1,1,38.0,female,1,0,71.2833,C,C
2,3,1,3,26.0,female,0,0,7.9250,N,S
3,4,1,1,35.0,female,1,0,53.1000,C,S
4,5,0,3,35.0,male,0,0,8.0500,N,S


In [4]:
n_cols = len(df.columns)
df.columns = ['id'] + ['y'] + ['x' + str(i) for i in range(n_cols - 2)]
df.columns


Index(['id', 'y', 'x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7'], dtype='object')

In [5]:
processed_df = Preprocessor().fit_transform(df)
processed_df

,id,y,x0,x1,x2,x3,x4,x5,x6,x7
0,1,0,3,22.0,1,1,0,7.2500,7,2
1,2,1,1,38.0,0,1,0,71.2833,2,0
2,3,1,3,26.0,0,0,0,7.9250,7,2
3,4,1,1,35.0,0,1,0,53.1000,2,2
4,5,0,3,35.0,1,0,0,8.0500,7,2
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,27.0,1,0,0,13.0000,7,2
887,888,1,1,19.0,0,0,0,30.0000,1,2
888,889,0,3,24.0,0,1,2,23.4500,7,2
889,890,1,1,26.0,1,0,0,30.0000,2,0


In [6]:
processed_df.to_csv("data/processed_titanic.csv")

In [7]:
num_data = processed_df.shape[0]
train_ratio = 0.2
train_df = processed_df[int(np.ceil(num_data * train_ratio)):]
valid_df = processed_df[:int(np.ceil(num_data * train_ratio))]

In [8]:
# train_df.to_csv('data/titanic_train.csv', index=False)
# valid_df.to_csv('data/titanic_valid.csv', index=False)

## Data split

In [9]:
# from tqdm import tqdm
# folders = ["8_0", "6_2", "4_4", "2_6", "0_8"]

# for folder in tqdm(folders):
    
#     if not os.path.exists(os.path.join("data", folder)):
#         os.mkdir(os.path.join("data", folder))
        
#     pos = int(folder.split("_")[0])
#     train_guest_df = train_df[train_df.columns[: pos + 2]]
#     train_host_df = train_df[['id'] + list(train_df.columns[pos + 2:])]
#     train_n_cols = len(train_host_df.columns)
#     train_host_df.columns = ['id'] + ['x' + str(i) for i in range(train_n_cols - 1)]
    
#     valid_guest_df = valid_df[valid_df.columns[: pos + 2]]
#     valid_host_df = valid_df[['id'] + list(valid_df.columns[pos + 2:])]
#     valid_n_cols = len(valid_host_df.columns)
#     valid_host_df.columns = ['id'] + ['x' + str(i) for i in range(valid_n_cols - 1)]

#     train_guest_df.to_csv(os.path.join("data", folder, 'train_guest.csv'), index=False)
#     train_host_df.to_csv(os.path.join("data", folder, 'train_host.csv'), index=False)
#     valid_guest_df.to_csv(os.path.join("data", folder, 'valid_guest.csv'), index=False)
#     valid_host_df.to_csv(os.path.join("data", folder, 'valid_host.csv'), index=False)
    
    

In [10]:
import os

In [11]:
!pipeline --help

Usage: pipeline [OPTIONS] COMMAND [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  config  pipeline config tool
  init     - DESCRIPTION: Pipeline Config Command.


Assume we have a `FATE Flow Service` in 127.0.0.1:9380(defaults in standalone), then exec

In [12]:
!pipeline init --ip 127.0.0.1 --port 9380

Pipeline configuration succeeded.


In [13]:
pwd

'/data/projects/fate/persistence/titanic-fl'

### Upload data

In [14]:
import os

In [15]:
!pipeline --help

Usage: pipeline [OPTIONS] COMMAND [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  config  pipeline config tool
  init     - DESCRIPTION: Pipeline Config Command.


Assume we have a `FATE Flow Service` which name is `fateflow` and the port is `9380`, then exec

In [16]:
!pipeline init --ip fateflow --port 9380

Pipeline configuration succeeded.


 Before start a modeling task, the data to be used should be uploaded. 
 Typically, a party is usually a cluster which include multiple nodes. Thus, when we upload these data, the data will be allocated to those nodes.

In [17]:
from pipeline.backend.pipeline import PipeLine

/data/projects/python/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Make a `pipeline` instance:

    - initiator: 
        * role: guest
        * party: 9999
    - roles:
        * guest: 9999

note that only local party id is needed.
    

In [18]:
pipeline_upload = PipeLine().set_initiator(role='guest', party_id=9999).set_roles(guest=9999)

Define partitions for data storage

In [19]:
partition = 4

Define table name and namespace, which will be used in FATE job configuration

In [20]:
# dense_data_guest = {"name": "titanic_guest", "namespace": f"experiment"}
# dense_data_host = {"name": "titanic_host", "namespace": f"experiment"}

dense_data_train_guest = {"name": "titanic_train_guest", "namespace": f"experiment"}
dense_data_train_host = {"name": "titanic_train_host", "namespace": f"experiment"}
dense_data_valid_guest = {"name": "titanic_valid_guest", "namespace": f"experiment"}
dense_data_valid_host  = {"name": "titanic_valid_host", "namespace": f"experiment"}

Now, we add data to be uploaded

In [21]:
folder = "2_6"

data_base = "/data/projects/fate/persistence/titanic-fl/data"
pipeline_upload.add_upload_data(file=os.path.join(data_base, folder, "train_guest.csv"),
                                table_name=dense_data_train_guest["name"],             # table name
                                namespace=dense_data_train_guest["namespace"],         # namespace
                                head=1, partition=partition)               # data info

pipeline_upload.add_upload_data(file=os.path.join(data_base, folder, "train_host.csv"),
                                table_name=dense_data_train_host["name"],
                                namespace=dense_data_train_host["namespace"],
                                head=1, partition=partition)

pipeline_upload.add_upload_data(file=os.path.join(data_base, folder, "valid_guest.csv"),
                                table_name=dense_data_valid_guest["name"],             # table name
                                namespace=dense_data_valid_guest["namespace"],         # namespace
                                head=1, partition=partition)               # data info

pipeline_upload.add_upload_data(file=os.path.join(data_base, folder, "valid_host.csv"),
                                table_name=dense_data_valid_host["name"],
                                namespace=dense_data_valid_host["namespace"],
                                head=1, partition=partition)

We can then upload data

In [22]:
pipeline_upload.upload(drop=1)

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%

2023-07-26 15:40:35.145 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202307261540350034700

2023-07-26 15:40:35.153 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00


m2023-07-26 15:40:36.168 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-07-26 15:40:36.169 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:01
2023-07-26 15:40:37.185 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:02
2023-07-26 15:40:38.200 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2023-07-26 15:40:39.215 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:04
2023-07-26 15:40:40.232 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:05
2023-07-26 15:40:41.246 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:06
2023-07-26 

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%

2023-07-26 15:40:42.410 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202307261540422649920

2023-07-26 15:40:42.416 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00


...

2023-07-26 15:40:53.730 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202307261540535913950

2023-07-26 15:40:53.737 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00


2023-07-26 15:40:54.746 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-07-26 15:40:55.761 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-07-26 15:40:55.762 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:02
2023-07-26 15:40:56.775 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2023-07-26 15:40:57.790 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:04
2023-07-26 15:40:58.804 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:05
2023-07-26 15:40:59.819 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:06
2023-07-26 15:41:0

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%

2023-07-26 15:41:01.985 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202307261541018473860

2023-07-26 15:41:01.992 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00


2023-07-26 15:41:03.001 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-07-26 15:41:04.016 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-07-26 15:41:04.017 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:02
2023-07-26 15:41:05.030 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2023-07-26 15:41:06.045 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:04
2023-07-26 15:41:07.059 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:05
2023-07-26 15:41:08.074 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:06
2023-07-26 15:41:0

## Modeling

### Local Model

In [68]:
train_df['y'].values

array([0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,

In [ ]:
criterion(ypred_var.squeeze(), y_var)

In [122]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)


In [132]:
target

tensor([4, 4, 0])

In [124]:
input.shape

torch.Size([3, 5])

In [128]:
train_df

,id,y,x0,x1,x2,x3,x4,x5,x6,x7
179,180,0,3,36.0,1,0,0,0.0000,7,2
180,181,0,3,24.0,0,8,2,69.5500,7,2
181,182,0,2,24.0,1,0,0,15.0500,7,0
182,183,0,3,9.0,1,4,2,31.3875,7,2
183,184,1,2,1.0,1,2,1,39.0000,5,2
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,27.0,1,0,0,13.0000,7,2
887,888,1,1,19.0,0,0,0,30.0000,1,2
888,889,0,3,24.0,0,1,2,23.4500,7,2
889,890,1,1,26.0,1,0,0,30.0000,2,0


In [129]:
train_df[train_df.columns[2:]]

,x0,x1,x2,x3,x4,x5,x6,x7
179,3,36.0,1,0,0,0.0000,7,2
180,3,24.0,0,8,2,69.5500,7,2
181,2,24.0,1,0,0,15.0500,7,0
182,3,9.0,1,4,2,31.3875,7,2
183,2,1.0,1,2,1,39.0000,5,2
...,...,...,...,...,...,...,...,...
886,2,27.0,1,0,0,13.0000,7,2
887,1,19.0,0,0,0,30.0000,1,2
888,3,24.0,0,1,2,23.4500,7,2
889,1,26.0,1,0,0,30.0000,2,0


In [130]:
train_df['y']

179    0
180    0
181    0
182    0
183    1
      ..
886    0
887    1
888    0
889    1
890    0
Name: y, Length: 712, dtype: int64

In [144]:
from sklearn.model_selection import train_test_split

x_train, y_train =  train_df[train_df.columns[2:]].values, train_df['y'].values

import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(8, 8)
        self.fc2 = nn.Linear(8, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
        
        return x
    
net = Net()

batch_size = 128
learning_rate = 0.001
batch_no = len(x_train) // batch_size
num_epochs = 1000
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

from sklearn.utils import shuffle
from torch.autograd import Variable

for epoch in range(num_epochs):
    if epoch % 5 == 0:
        print('Epoch {}'.format(epoch+1))
    x_train, y_train = shuffle(x_train, y_train)
    
    # Mini batch learning
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size
        x_var = Variable(torch.FloatTensor(x_train[start:end]))
        y_var = Variable(torch.FloatTensor(y_train[start:end]))
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        ypred_var = net(x_var)
        loss = criterion(ypred_var.squeeze(), y_var)
        loss.backward()
        optimizer.step()
        
        
# Evaluate the model
test_var = Variable(torch.FloatTensor(x_train), requires_grad=True)
with torch.no_grad():
    result = net(test_var)
values, labels = torch.max(result, 1)
# num_right = np.sum(labels.data.numpy() == y_val)
# print('Accuracy {:.2f}'.format(num_right / len(y_val)))


import numpy as np
from sklearn import metrics

y_true = y_train

threshold = 0.5

# 將機率值轉換成二元的 0 和 1
y_pred = (values >= threshold).int()

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred)
metrics.auc(fpr, tpr)

Epoch 1
Epoch 6
Epoch 11
Epoch 16
Epoch 21
Epoch 26
Epoch 31
Epoch 36
Epoch 41
Epoch 46
Epoch 51


/data/projects/python/venv/lib/python3.8/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 56
Epoch 61
Epoch 66
Epoch 71
Epoch 76
Epoch 81
Epoch 86
Epoch 91
Epoch 96
Epoch 101
Epoch 106
Epoch 111
Epoch 116
Epoch 121
Epoch 126
Epoch 131
Epoch 136
Epoch 141
Epoch 146
Epoch 151
Epoch 156
Epoch 161
Epoch 166
Epoch 171
Epoch 176
Epoch 181
Epoch 186
Epoch 191
Epoch 196
Epoch 201
Epoch 206
Epoch 211
Epoch 216
Epoch 221
Epoch 226
Epoch 231
Epoch 236
Epoch 241
Epoch 246
Epoch 251
Epoch 256
Epoch 261
Epoch 266
Epoch 271
Epoch 276
Epoch 281
Epoch 286
Epoch 291
Epoch 296
Epoch 301
Epoch 306
Epoch 311
Epoch 316
Epoch 321
Epoch 326
Epoch 331
Epoch 336
Epoch 341
Epoch 346
Epoch 351
Epoch 356
Epoch 361
Epoch 366
Epoch 371
Epoch 376
Epoch 381
Epoch 386
Epoch 391
Epoch 396
Epoch 401
Epoch 406
Epoch 411
Epoch 416
Epoch 421
Epoch 426
Epoch 431
Epoch 436
Epoch 441
Epoch 446
Epoch 451
Epoch 456
Epoch 461
Epoch 466
Epoch 471
Epoch 476
Epoch 481
Epoch 486
Epoch 491
Epoch 496
Epoch 501
Epoch 506
Epoch 511
Epoch 516
Epoch 521
Epoch 526
Epoch 531
Epoch 536
Epoch 541
Epoch 546
Epoch 551
Epoch 556

0.8180500300641642

In [103]:
y_var.shape

torch.Size([50])

In [23]:
import argparse
from sklearn.metrics import accuracy_score

import torch as t
from torch import nn

from pipeline import fate_torch_hook
from pipeline.backend.pipeline import PipeLine
from pipeline.component import DataTransform
from pipeline.component import HeteroSecureBoost, HeteroLR, HeteroNN
from pipeline.component import Intersection
from pipeline.component import Reader
from pipeline.interface import Data
from pipeline.component import Evaluation
from pipeline.interface import Model

from pipeline.utils.tools import load_job_config

In [24]:
guest, host, arbiter = 9999, 10000, 10000

# init pipeline
pipeline = PipeLine().set_initiator(role="guest", party_id=guest).set_roles(guest=guest, host=host, arbiter=arbiter)

In [25]:
# set data reader and data-io
reader_0, reader_1 = Reader(name="reader_0"), Reader(name="reader_1")
reader_0.get_party_instance(role="guest", party_id=guest).component_param(table=dense_data_train_guest)
reader_0.get_party_instance(role="host", party_id=host).component_param(table=dense_data_train_host)
# reader_1.get_party_instance(role="guest", party_id=guest).component_param(table=dense_data_valid_guest)
# reader_1.get_party_instance(role="host", party_id=host).component_param(table=dense_data_valid_host)

In [26]:
data_transform_0, data_transform_1 = DataTransform(name="data_transform_0"), DataTransform(name="data_transform_1")

data_transform_0.get_party_instance(
    role="guest", party_id=guest).component_param(
    with_label=True, output_format="dense")
data_transform_0.get_party_instance(role="host", party_id=host).component_param(with_label=False)
# data_transform_1.get_party_instance(
#     role="guest", party_id=guest).component_param(
#     with_label=True, output_format="dense")
# data_transform_1.get_party_instance(role="host", party_id=host).component_param(with_label=False)

In [27]:
# data intersect component
intersect_0 = Intersection(name="intersection_0")
# intersect_1 = Intersection(name="intersection_1")

pipeline.add_component(reader_0)
pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
pipeline.add_component(intersect_0, data=Data(data=data_transform_0.output.data))

### HeteroNN

In [28]:
t = fate_torch_hook(t)
guest_n_feature, host_n_feature = folder.split("_")
guest_n_feature, host_n_feature = int(guest_n_feature), int(host_n_feature)

hetero_nn_0 = HeteroNN(name="hetero_nn_0", epochs=20,
                       interactive_layer_lr=0.01, batch_size=-1, validation_freqs=1, task_type='classification', early_stop="diff")
guest_nn_0 = hetero_nn_0.get_party_instance(role='guest', party_id=guest)
host_nn_0 = hetero_nn_0.get_party_instance(role='host', party_id=host)

# define model
guest_bottom = t.nn.Sequential(
    nn.Linear(guest_n_feature, 8),
    nn.ReLU()
)

guest_top = t.nn.Sequential(
    nn.ReLU(),
    nn.Linear(8, 1),
    nn.Sigmoid()
)

host_bottom = t.nn.Sequential(
    nn.Linear(host_n_feature, 8),
    nn.ReLU()
)

# use interactive layer after fate_torch_hook
interactive_layer = t.nn.InteractiveLayer(out_dim=8, guest_dim=8, host_dim=8, host_num=1)

guest_nn_0.add_top_model(guest_top)
guest_nn_0.add_bottom_model(guest_bottom)
host_nn_0.add_bottom_model(host_bottom)

optimizer = t.optim.Adam(lr=0.01)  # you can initialize optimizer without parameters after fate_torch_hook
loss = t.nn.BCELoss()

hetero_nn_0.set_interactive_layer(interactive_layer)
hetero_nn_0.compile(optimizer=optimizer, loss=loss)

evaluation_0 = Evaluation(name='eval_0', eval_type='binary')

# define components IO
pipeline.add_component(hetero_nn_0, data=Data(train_data=intersect_0.output.data))
pipeline.add_component(evaluation_0, data=Data(data=hetero_nn_0.output.data))
pipeline.compile()
pipeline.fit()

2023-07-26 15:41:10.624 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202307261541101603700

2023-07-26 15:41:10.632 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2023-07-26 15:41:11.641 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-07-26 15:41:12.657 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-07-26 15:41:12.658 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:02
2023-07-26 15:41:13.673 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2023-07-26 15:41:14.688 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2023-07-26 15:41:15.702 | INFO     | pipel

2023-07-26 15:41:47.213 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:36
2023-07-26 15:41:48.228 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:37
2023-07-26 15:41:49.243 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:38
2023-07-26 15:41:50.263 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:39
2023-07-26 15:41:51.278 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:40
2023-07-26 15:41:52.312 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:41
2023-07-26 15:41:53.331 | INFO     | pipeline.utils.invoker.job_submitter:mo

2023-07-26 15:42:24.849 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:14
2023-07-26 15:42:25.863 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:15
2023-07-26 15:42:26.879 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:16
2023-07-26 15:42:27.901 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:17
2023-07-26 15:42:28.922 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:18
2023-07-26 15:42:29.955 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:19
2023-07-26 15:42:30.976 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2023-07-26 15:43:04.491 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:53
2023-07-26 15:43:05.508 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:54
2023-07-26 15:43:06.522 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:55
2023-07-26 15:43:07.538 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:56
2023-07-26 15:43:08.553 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:57
2023-07-26 15:43:09.568 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:58
2023-07-26 15:43:10.586 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2023-07-26 15:43:44.106 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:33
2023-07-26 15:43:45.121 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:34
2023-07-26 15:43:46.137 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:35
2023-07-26 15:43:47.152 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:36
2023-07-26 15:43:48.166 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:37
2023-07-26 15:43:49.182 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:02:38
2023-07-26 15:43:50.197 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

...

2023-07-26 15:58:15.444 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:04
2023-07-26 15:58:16.458 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:05
2023-07-26 15:58:17.472 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:06
2023-07-26 15:58:18.487 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:07
2023-07-26 15:58:19.510 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:08
2023-07-26 15:58:20.525 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:09
2023-07-26 15:58:21.547 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2023-07-26 15:58:55.066 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:44
2023-07-26 15:58:56.080 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:45
2023-07-26 15:58:57.096 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:46
2023-07-26 15:58:58.110 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:47
2023-07-26 15:58:59.125 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:48
2023-07-26 15:59:00.140 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:17:49
2023-07-26 15:59:01.156 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2023-07-26 15:59:34.671 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:18:24
2023-07-26 15:59:35.685 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:18:25
2023-07-26 15:59:36.705 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:18:26
2023-07-26 15:59:37.720 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:18:27
2023-07-26 15:59:38.735 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:18:28
2023-07-26 15:59:39.751 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:18:29
2023-07-26 15:59:40.767 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2023-07-26 16:00:14.293 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:19:03
2023-07-26 16:00:15.310 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:19:04
2023-07-26 16:00:16.326 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:19:05
2023-07-26 16:00:17.342 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:19:06
2023-07-26 16:00:18.357 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:19:07
2023-07-26 16:00:19.372 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:19:08
2023-07-26 16:00:20.388 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2023-07-26 16:00:52.883 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component eval_0, time elapse: 0:19:42
2023-07-26 16:00:53.900 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component eval_0, time elapse: 0:19:43
2023-07-26 16:00:55.923 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:89 - Job is success!!! Job id is 202307261541101603700
2023-07-26 16:00:55.924 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:90 - Total time: 0:19:45


## Evaluation

In [29]:
print("fitting hetero nn done, result:")
print(pipeline.get_component("hetero_nn_0").get_summary())

print('start to predict')

# predict
# deploy required components
pipeline.deploy_component([data_transform_0, intersect_0, hetero_nn_0, evaluation_0])

predict_pipeline = PipeLine()
# add data reader onto predict pipeline
predict_pipeline.add_component(reader_0)
# add selected components from train pipeline onto predict pipeline
# specify data source
predict_pipeline.add_component(
    pipeline, data=Data(
        predict_input={
            pipeline.data_transform_0.input.data: reader_0.output.data}))

# run predict model
predict_pipeline.predict()
predict_result = predict_pipeline.get_component("hetero_nn_0").get_output_data()

fitting hetero nn done, result:
{'best_iteration': -1, 'history_loss': [0.6668531894683838, 0.6360350251197815, 0.6220659613609314, 0.606635570526123, 0.5978214144706726, 0.5939688682556152, 0.5908780694007874, 0.587712287902832, 0.5857213139533997, 0.5842297673225403, 0.5826376676559448, 0.5809691548347473, 0.5793160796165466, 0.5776740312576294, 0.5763044953346252, 0.5751323699951172, 0.574042558670044, 0.5729953646659851, 0.5720847249031067, 0.5711174011230469], 'is_converged': False, 'validation_metrics': {'train': {'auc': [0.7158977653677301, 0.720308548930457, 0.7235867783570964, 0.7273592132249376, 0.733297915276714, 0.7390801189387762, 0.7419671023911307, 0.7440921857882989, 0.7467732503068193, 0.748437075292199, 0.7503768316489163, 0.7527778464174224, 0.7549770606307709, 0.7571186175426458, 0.7583005922228538, 0.7595443425832118, 0.7608004480795999, 0.7622830644032057, 0.763032609322362, 0.7641280980503595], 'ks': [0.3714365728500005, 0.381353628703452, 0.381353628703452, 0.38

2023-07-26 16:00:59.823 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202307261600592109750

2023-07-26 16:00:59.830 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
m2023-07-26 16:01:00.848 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-07-26 16:01:00.849 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:01
2023-07-26 16:01:01.863 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:02
2023-07-26 16:01:02.878 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2023-07-26 16:01:03.892 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2023-07-26 16:01:04.906 | INFO     

m2023-07-26 16:01:36.449 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-07-26 16:01:36.450 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:36
2023-07-26 16:01:37.465 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:37
2023-07-26 16:01:38.479 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:38
2023-07-26 16:01:39.493 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:39
2023-07-26 16:01:40.507 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:40
2023-07-26 16:01:41.532 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection

2023-07-26 16:02:14.095 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:14
2023-07-26 16:02:15.120 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:15
2023-07-26 16:02:16.137 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:16
2023-07-26 16:02:17.154 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:17
2023-07-26 16:02:18.175 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:18
2023-07-26 16:02:19.197 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:19
2023-07-26 16:02:20.215 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:1

2023-07-26 16:02:53.751 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:53
2023-07-26 16:02:54.766 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:54
2023-07-26 16:02:55.792 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:55
2023-07-26 16:02:56.808 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:56
2023-07-26 16:02:57.824 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:58
2023-07-26 16:02:58.841 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_nn_0, time elapse: 0:01:59
m2023-07-26 16:02:59.859 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:

In [30]:
pipeline.get_component('reader_0').get_output_data(limits=10)

,id,y,x0,x1
0,577,1,2,34.0
1,578,1,1,39.0
2,579,0,3,24.0
3,580,1,3,32.0
4,581,1,2,25.0
5,582,1,1,39.0
6,583,0,2,54.0
7,584,0,1,36.0
8,585,0,3,24.0


In [31]:
pipeline.get_component("hetero_nn_0").get_summary()

{'best_iteration': -1,
 'history_loss': [0.6668531894683838,
  0.6360350251197815,
  0.6220659613609314,
  0.606635570526123,
  0.5978214144706726,
  0.5939688682556152,
  0.5908780694007874,
  0.587712287902832,
  0.5857213139533997,
  0.5842297673225403,
  0.5826376676559448,
  0.5809691548347473,
  0.5793160796165466,
  0.5776740312576294,
  0.5763044953346252,
  0.5751323699951172,
  0.574042558670044,
  0.5729953646659851,
  0.5720847249031067,
  0.5711174011230469],
 'is_converged': False,
 'validation_metrics': {'train': {'auc': [0.7158977653677301,
    0.720308548930457,
    0.7235867783570964,
    0.7273592132249376,
    0.733297915276714,
    0.7390801189387762,
    0.7419671023911307,
    0.7440921857882989,
    0.7467732503068193,
    0.748437075292199,
    0.7503768316489163,
    0.7527778464174224,
    0.7549770606307709,
    0.7571186175426458,
    0.7583005922228538,
    0.7595443425832118,
    0.7608004480795999,
    0.7622830644032057,
    0.763032609322362,
    0.764

In [32]:
predict_result.head()

,id,label,predict_result,predict_score,predict_detail,type
0,332,0.0,1,0.5069051384925842,"{'0': 0.49309486150741577, '1': 0.506905138492...",predict
1,793,0.0,0,0.3694379925727844,"{'0': 0.6305620074272156, '1': 0.3694379925727...",predict
2,391,1.0,1,0.782852828502655,"{'0': 0.21714717149734497, '1': 0.782852828502...",predict
3,373,0.0,0,0.25969773530960083,"{'0': 0.7403022646903992, '1': 0.2596977353096...",predict
4,672,0.0,1,0.6389840245246887,"{'0': 0.3610159754753113, '1': 0.6389840245246...",predict


In [33]:
y_true = predict_result['label'].astype(float).astype(int).copy()
y_pred = predict_result['predict_result'].astype(int).copy()

In [34]:
## Calculate Accuracy
# y_true, y_pred = predict_result['label'], predict_result['predict_result']
acc = accuracy_score(y_true, y_pred)
print("accuary: ", acc)

accuary:  0.7064606741573034
